# Convolution As an Image Filter
Justin Reising




In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np 

from scipy.signal import convolve2d, convolve, fftconvolve
from scipy.fftpack import fft2, ifft2, dctn, idctn
from skimage import color
np.set_printoptions( precision = 5, suppress = True, linewidth = 100 )
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
x = plt.imread('etsuentrance.jpg')
x.shape

Because the image is a tensor, we flatten it into a matrix by converting to grayscale. The convolution filters operate in the same manner regardless of the dimensions. However, flattening to a matrix simplifies calculations and improves calculation time. 

In [ ]:
x = color.rgb2gray(x)
x.shape

In [ ]:
plt.figure( figsize = (8,4) )  #15.5,6))
plt.gray() # plot image in grayscale
plt.imshow(x);
plt.axis('off')

The convolution filters are defined by referencing the website: $$http://setosa.io/ev/image-kernels$$. We then verify that the results match the examples given in the tutorial.


In [ ]:
h = {     'sharpen': np.array([[ 0, -1,  0],
                               [-1,  5, -1],
                               [ 0, -1,  0]]), 
       'left_sobel': np.array([[1, 0, -1],
                               [2, 0, -2],
                               [1, 0, -1]]),
      'bottom_sobel':np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]]),
       'right_sobel':np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]]), 
      'EdgeDetector':np.array([[-1, -1, -1],
                               [-1,  8, -1],
                               [-1, -1, -1]]),
         'Laplacian':np.array([[ 0, -1,  0],
                               [-1,  4, -1],
                               [ 0, -1,  0]]),
       'top_sobel':np.array([[1,2,1],
                           [0,0,0],
                           [-1,-2,-1]]),
     'emboss' :np.array([[-2,-1,0],
                        [-1,1,1],
                        [0,1,2]]),
     'blur' :np.array([[0.0625,0.125,0.0625],
                      [0.125,0.25,0.125],
                      [0.0625,0.125,0.0625]])
     
    }  

In [ ]:
%time ydirect = convolve2d(h['left_sobel'],x)

In [ ]:
plt.figure( figsize =  (8,4) )
plt.imshow(ydirect);
plt.axis('off')

In order to accelerate calculation, the properties of the Fast Fourier Transform, and convolution are applied. Within the $Scipy.Signal$ library, the "$fftconvolve$" command will be used in lieu of the standard convolution. Observe that both operations produce the same image after applying the filter, but the convolution involving the Fast Fourier Transform is markedly faster.

In [ ]:
%time yfft = fftconvolve(h['left_sobel'],x)

In [ ]:
plt.figure( figsize =  (8,4) )
plt.imshow(yfft);
plt.axis('off')

In [ ]:
np.allclose(ydirect, yfft)

Note that the size of the image has changed after applying the filter. this is because the filter is a $3x3$matrix, and as such produces some "excess" pixels when it is applied to the edges of the image. This be alleviated by using the Fast Fourier Transform to create an image composed of four copies of the original image and slicing the original image out.

In [ ]:
%time yfft = fftconvolve(h['bottom_sobel'],x, )

In [ ]:
plt.figure( figsize =  (8,4) )
plt.imshow(yfft);
plt.axis('off')

In [ ]:
fft2(x)[:5,:3]

In [ ]:
fft2(x)[:5,:3]

In [ ]:
xrev = x[:,::-1]

In [ ]:
plt.figure( figsize =  (8,4) )
plt.imshow(xrev);
plt.axis('off')

In [ ]:
xext_x = np.hstack([x,xrev])
plt.figure( figsize =  (16,4) )
plt.imshow(xext_x);
plt.axis('off')

In [ ]:
x_sym = np.vstack([xext_x, xext_x[::-1,:]] )

In [ ]:
plt.figure( figsize =  (8,8) )
plt.imshow(x_sym);
plt.axis('off')

In [ ]:
fft2(x_sym )[:5,:3]

In [ ]:
dctn(x)[:5,:3]

In [ ]:
yedges = fftconvolve(h['EdgeDetector'], x )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(yedges)
axes[0].axis('off')
axes[0].set_title('Using Edge Filter')
axes[1].imshow(x)
axes[1].axis('off')
axes[1].set_title('Original Image');


Once the images are passed through a filter, is it possible to reconstruct the original from the filtered result? In order to come to a conclusion on this matter, we apply the two-dimensional Fast Fourier Transform to the convolution operator, and convolve it with the filtered image. Then, the inverse two-dimensional Fast Fourier Transform is applied.

In [ ]:
H = fft2(h['EdgeDetector'])
H

In [ ]:
Hinv = 1/H 
Hinv[0,0] = 0
Hinv

In [ ]:
hinv = ifft2( Hinv )
hinv

In [ ]:
x_rec = fftconvolve( hinv, yedges)
x_rec[:5, :3]

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(x_rec.real )
axes[0].axis('off')
axes[0].set_title('Reconstructed from Edge Detected')
axes[1].imshow(x)
axes[1].axis('off')
axes[1].set_title("Original Image -- Yeah, Inversion Doesn't Work!");

From above, we note that this inversion technique is not effective in reverting the filtered image back to the original. 

In [ ]:
M,N = x.shape
yedges = fftconvolve(h['Laplacian'], x )/(M-1)/(N-1)

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(yedges)
axes[0].axis('off')
axes[0].set_title('Using Laplacian Edge Filter (2nd derivative)')
axes[1].imshow(x)
axes[1].axis('off')
axes[1].set_title('Original Image');


Convolution image filters are not limited in dimension to $3x3$ matrix operators. To illustrate, we define several $5x5$ convolution operators and compare and contrast between the two dimensions.

In [ ]:
five = {     
       'left_sobel':np.array([[ 1,  2,  0,  -2, -1],
                 [ 4, 8, 0, -8, -4],
                 [ 6, 12, 0, -12,-6],
                 [ 4, 8, 0, -8, -4],
                 [ 1,  2,  0,  -2, -1] ]),
      'bottom_sobel':np.array([[ -1,  -4,  -6,  -4, -1],
                 [ -2, -8, -12, -8, -2],
                 [ 0, 0, 0, 0,0],
                 [ 2, 8, 12, 8, 2],
                   [ 1,  4,  6,  4, 1]]),
       'right_sobel':np.array([[ -1,  -2,  0,  2, 1],
                 [ -4, -8, 0, 8, 4],
                 [ -6, -12, 0, 12,6],
                 [ -4, -8, 0, 8, 4],
                 [ -1,  -2,  0,  2, 1] ]), 
       'top_sobel':np.array([[ 1,  4,  6,  4, 1],
                 [ 2, 8, 12, 8, 2],
                 [ 0, 0, 0, 0,0],
                 [ -2, -8, -12, -8, -2],
                 [ -1,  -4,  -6,  -4, -1] ]),
    'Blur':np.array([[ 1,  4,  6,  4, 1],
                 [ 4, 16, 24, 16, 4],
                 [ 6, 24, 36, 24, 6],
                 [ 4, 16, 24, 16, 4],
                 [ 1,  4,  6,  4, 1]]) / 256
     
    }

In [ ]:
yedges = fftconvolve(five['Blur'], x )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(yedges)
axes[0].axis('off')
axes[0].set_title('Gaussian Blurred')
axes[1].imshow(x)
axes[1].axis('off')
axes[1].set_title('Original Image');


In [ ]:
filt=fftconvolve(five['bottom_sobel'],x, )
botthree=fftconvolve(h['bottom_sobel'],x, )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(filt)
axes[0].axis('off')
axes[0].set_title('Bottom Sobel(5x5)')
axes[1].imshow(botthree)
axes[1].axis('off')
axes[1].set_title('Bottom Sobel(3x3)');


In [ ]:
filt=fftconvolve(five['top_sobel'],x, )
topthree=fftconvolve(h['top_sobel'],x, )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(filt)
axes[0].axis('off')
axes[0].set_title('Top Sobel (5x5)')
axes[1].imshow(topthree)
axes[1].axis('off')
axes[1].set_title('Top Sobel (3x3)');

From the figure above, we note that the $5x5$ operator produces an image that is marginally more blurred than the $3x3$ operator. To verify this behavior, we import a separate image with larger dimensions (i.e. the ETSU Minidome image) and compare the results further.

In [ ]:
y = plt.imread('ETSUminidomeFront.jpg')
y.shape

In [ ]:
y = color.rgb2gray(y)
y.shape

In [ ]:
plt.figure( figsize = (8,4) )  #15.5,6))
plt.gray() # plot image in grayscale
plt.imshow(y);
plt.axis('off')

In [ ]:
mini=fftconvolve(five['top_sobel'],y, )
mini3=fftconvolve(h['top_sobel'],y, )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(mini)
axes[0].axis('off')
axes[0].set_title('Top Sobel (5x5)')
axes[1].imshow(mini3)
axes[1].axis('off')
axes[1].set_title('Top Sobel (3x3)');

Observe here that the larger image filter produced an image that is lighter, and that the text is easier to distinguish and more legible. 

In [ ]:
mini=fftconvolve(five['bottom_sobel'],y, )
mini3=fftconvolve(h['bottom_sobel'],y, )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(mini)
axes[0].axis('off')
axes[0].set_title('Bottom Sobel (5x5)')
axes[1].imshow(mini3)
axes[1].axis('off')
axes[1].set_title('Bottom Sobel (3x3)');

In [ ]:
mini=fftconvolve(five['right_sobel'],y, )
mini3=fftconvolve(h['right_sobel'],y, )

In [ ]:
fig, axes = plt.subplots( nrows = 2, ncols = 1, figsize =  (8,8) )
axes[0].imshow(mini)
axes[0].axis('off')
axes[0].set_title('Right Sobel (5x5)')
axes[1].imshow(mini3)
axes[1].axis('off')
axes[1].set_title('Right Sobel (3x3)');

As the larger convolution produces more definition after filtering for the above operators, it can be surmised that smaller operators are better for smaller images, while larger images require larger filter matrices. 